In [1]:
import pandas as pd
import pandas
from pandas import Series

In [2]:
#Read in CCH data and calculate DOY for each specimen
### note that for much of the CCH Data, phenological data is only available through direct request to the jepson herbarium
### and at the time of this publication, is not directly available through any web-accessible portal

baseData = pd.read_csv("CCH_Data.csv") # insert name of CCH Data here


/Users/isaacpark/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print len(baseData)

201


In [4]:
#Convert raw Year-Month-Day into three separate fields in order to calculate DOY
DOYSplitter = baseData['early_date'].str.split(' ').apply(Series, 1).stack()
DOYSplitter.index = DOYSplitter.index.droplevel(-1) #modify DOYSplitter to line up with df's index

baseData = baseData.join(DOYSplitter.apply(lambda x: Series(x.split('-'))))
baseData['year'] = baseData[0].astype(float)
baseData['month'] = baseData[1].astype(float)
baseData['day'] = baseData[2].astype(float)

del baseData[0]
del baseData[1]
del baseData[2]
print len(baseData)
baseData.head()

201


,id,scientificName,collectors,collector_number,early_date,late_date,verbatim_date,county,elevation,locality,...,population_biology,phenology,associates,biotic_interactions,fruit_removed,material_removed,other,year,month,day
0,CAS491913,Lessingia leptoclada,Willard Spence,733,1962-07-25,1962-07-25,25 July 1962,El Dorado,NaN,"3 miles east of Auburn, Route 49 on the road t...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1962.0,7.0,25.0
1,CDA40187,Psilocarphus tenellus,"G. F. Hrusa, R. Gross, G. Holstein",18546,2014-05-14,2014-05-14,May 14 2014,Lake,1450 ft,Silver Spur Ranch. Benmore Canyon at intersect...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0,5.0,14.0
2,DS708828,Erigeron coulteri,Burt Walker,42,1930-08-04,1930-08-04,"August 4, 1930.",Tuolumne,7500 ft,"Pate Valley Trail, 1 mile from Harden Lake, Yo...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1930.0,8.0,4.0
3,HSC91853,Ehrharta calycina,Susan H. Bicknell,,1988-08-20,1988-08-20,Aug 20 1988,San Luis Obispo,NaN,Motana de Oro Park,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988.0,8.0,20.0
4,LA52255,Eriogonum fasciculatum var. fasciculatum,Carl Epling,6506,1925-03-01,1925-03-31,March 1925,Los Angeles,NaN,"SANTA MONICA MOUNTAINS, Griffith Park",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1925.0,3.0,1.0


In [5]:
#Calculate whether year of collection for each specimen was a leap year 
#- assign new data column value of '1' if so, or '0' if not

baseData["LeapYear"] = 0
def LeapYearCalc(row):
    leapYear = 0
    if row['year'] % 4 ==0:
        leapYear = 1
        if row['year'] % 100 == 0:
            leapYear = 0
        if row['year'] % 400 == 0:
            leapYear = 1
    return leapYear


baseData["LeapYear"] = baseData.apply(LeapYearCalc, axis=1) 
baseData.head()

,id,scientificName,collectors,collector_number,early_date,late_date,verbatim_date,county,elevation,locality,...,phenology,associates,biotic_interactions,fruit_removed,material_removed,other,year,month,day,LeapYear
0,CAS491913,Lessingia leptoclada,Willard Spence,733,1962-07-25,1962-07-25,25 July 1962,El Dorado,NaN,"3 miles east of Auburn, Route 49 on the road t...",...,NaN,NaN,NaN,NaN,NaN,NaN,1962.0,7.0,25.0,0
1,CDA40187,Psilocarphus tenellus,"G. F. Hrusa, R. Gross, G. Holstein",18546,2014-05-14,2014-05-14,May 14 2014,Lake,1450 ft,Silver Spur Ranch. Benmore Canyon at intersect...,...,NaN,NaN,NaN,NaN,NaN,NaN,2014.0,5.0,14.0,0
2,DS708828,Erigeron coulteri,Burt Walker,42,1930-08-04,1930-08-04,"August 4, 1930.",Tuolumne,7500 ft,"Pate Valley Trail, 1 mile from Harden Lake, Yo...",...,NaN,NaN,NaN,NaN,NaN,NaN,1930.0,8.0,4.0,0
3,HSC91853,Ehrharta calycina,Susan H. Bicknell,,1988-08-20,1988-08-20,Aug 20 1988,San Luis Obispo,NaN,Motana de Oro Park,...,NaN,NaN,NaN,NaN,NaN,NaN,1988.0,8.0,20.0,1
4,LA52255,Eriogonum fasciculatum var. fasciculatum,Carl Epling,6506,1925-03-01,1925-03-31,March 1925,Los Angeles,NaN,"SANTA MONICA MOUNTAINS, Griffith Park",...,NaN,NaN,NaN,NaN,NaN,NaN,1925.0,3.0,1.0,0


In [6]:
#Calculate DOY of collection for each specimen

def DOYCalc(row):
    
    if row['month'] == 1:
        return row['day']
    elif row['month'] ==2:
        return row['day'] + 31
    elif row['month'] ==3:
        return row['day'] + 59 + row['LeapYear']
    elif row['month'] ==4:
        return row['day'] + 90 + row['LeapYear']
    elif row['month'] == 5:
        return row['day'] + 120+ row['LeapYear']
    elif row['month'] == 6:
        return row['day']  + 151 + row['LeapYear']
    elif row['month'] == 7:
        return row['day']  + 181 + row['LeapYear']
    elif row['month'] == 8:
        return row['day'] + 212 + row['LeapYear']
    elif row['month'] == 9:
        return row['day']  + 243 + row['LeapYear']
    elif row['month'] == 10:
        return row['day']  + 273 + row['LeapYear']
    elif row['month'] == 11:
        return row['day']  + 304 + row['LeapYear']
    elif row['month'] == 12:
        return row['day']  + 334 + row['LeapYear']

    
baseData["DOY"] = baseData.apply(DOYCalc, axis=1) 
baseData.head()

,id,scientificName,collectors,collector_number,early_date,late_date,verbatim_date,county,elevation,locality,...,associates,biotic_interactions,fruit_removed,material_removed,other,year,month,day,LeapYear,DOY
0,CAS491913,Lessingia leptoclada,Willard Spence,733,1962-07-25,1962-07-25,25 July 1962,El Dorado,NaN,"3 miles east of Auburn, Route 49 on the road t...",...,NaN,NaN,NaN,NaN,NaN,1962.0,7.0,25.0,0,206.0
1,CDA40187,Psilocarphus tenellus,"G. F. Hrusa, R. Gross, G. Holstein",18546,2014-05-14,2014-05-14,May 14 2014,Lake,1450 ft,Silver Spur Ranch. Benmore Canyon at intersect...,...,NaN,NaN,NaN,NaN,NaN,2014.0,5.0,14.0,0,134.0
2,DS708828,Erigeron coulteri,Burt Walker,42,1930-08-04,1930-08-04,"August 4, 1930.",Tuolumne,7500 ft,"Pate Valley Trail, 1 mile from Harden Lake, Yo...",...,NaN,NaN,NaN,NaN,NaN,1930.0,8.0,4.0,0,216.0
3,HSC91853,Ehrharta calycina,Susan H. Bicknell,,1988-08-20,1988-08-20,Aug 20 1988,San Luis Obispo,NaN,Motana de Oro Park,...,NaN,NaN,NaN,NaN,NaN,1988.0,8.0,20.0,1,233.0
4,LA52255,Eriogonum fasciculatum var. fasciculatum,Carl Epling,6506,1925-03-01,1925-03-31,March 1925,Los Angeles,NaN,"SANTA MONICA MOUNTAINS, Griffith Park",...,NaN,NaN,NaN,NaN,NaN,1925.0,3.0,1.0,0,60.0
